## Combining the individual .xlsx files into bigger batches of .xlsx files

In [19]:
#Import necessary library
import pandas as pd
import os
import requests
import urllib
import xlrd
import re

In [20]:
# Append the directory in the ScrapedData sub-folder
base = './scrapedData/'
tempList = os.listdir('scrapedData')
fileList = list()
for file in tempList:
    fileList += [base + file]

# display individual .xlsx names in the ScrapedData sub-folder
fileList

['./scrapedData/scrapingData_ACJC-Confessions-365341810240423_FULL.xlsx',
 './scrapedData/scrapingData_andiechen_FULL.xlsx',
 './scrapedData/scrapingData_asrjcconfessions_FULL.xlsx',
 './scrapedData/scrapingData_bellywellyjelly_FULL.xlsx',
 './scrapedData/scrapingData_benjamin.kheng_FULL.xlsx',
 './scrapedData/scrapingData_bossyflossie_FULL.xlsx',
 './scrapedData/scrapingData_cjcroxx_FULL.xlsx',
 './scrapedData/scrapingData_DanielFoodDiary_FULL.xlsx',
 './scrapedData/scrapingData_DHS-Confessions-103690209814932_FULL.xlsx',
 './scrapedData/scrapingData_DollarsAndSenseSG_FULL.xlsx',
 './scrapedData/scrapingData_dreachongofficial_FULL.xlsx',
 './scrapedData/scrapingData_HwaChongConfessions_FULL.xlsx',
 './scrapedData/scrapingData_ieatishootipost_FULL.xlsx',
 './scrapedData/scrapingData_InnovaConfessions_FULL.xlsx',
 './scrapedData/scrapingData_ITE-College-Central-Confessions-102332676616681_FULL.xlsx',
 './scrapedData/scrapingData_ITE-College-West-Confessions-123845157793064_FULL.xlsx',
 

In [13]:
# Group the scraped data into small batches to ensure that the total number of lines 
# in each batch of text does not exceed 65000 ()
#finalList[:34]
#finalList[34:]

In [17]:
#Create a dataframe
df = pd.DataFrame()

# read them in
excels = [pd.ExcelFile(item) for item in fileList]

# turn them into dataframes
frames = [x.parse(x.sheet_names[0], header=None,index_col=None) for x in excels]

# delete the first row for all frames except the first
# i.e. remove the header row -- assumes it's the first
frames[1:] = [df[1:] for df in frames[1:]]

# concatenate them..
combined = pd.concat(frames)

In [18]:
#Save the output of the combined dataset into one excel
combined.to_excel("combinedScrapedData.xlsx", header=False, index=False)

## Cleaning of Data

In [ ]:
# check number of rows of text in the .xlsx file
file = base + "combinedScrapedData.xlsx"